In [ ]:
import os
import sys
from functools import partial
import warnings

import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
%matplotlib inline
import seaborn as sns

sys.path.append('../')
from src.utils import parallel_apply
from src.feature_extraction import add_features_in_group

warnings.filterwarnings('ignore')

DIR = '/PATH/TO/YOUR/DATA'
description = pd.read_csv(os.path.join(DIR,'data/HomeCredit_columns_description.csv'),encoding = 'latin1')
application = pd.read_csv(os.path.join(DIR, 'files/unzipped_data/application_train.csv'))
bureau = pd.read_csv(os.path.join(DIR, 'files/unzipped_data/bureau.csv'))

In [ ]:
bureau.head()

# Preprocessing
## Solution 3

In [ ]:
(bureau['AMT_CREDIT_SUM'] == 0).sum()

This shows that imputing with nan with 0 is probably a bad idea

# Feature Engineering
## Solution 3
### Hand crafted features

In [ ]:
bureau[bureau['SK_ID_CURR']==215354]

### First build helper columns

In [ ]:
bureau['bureau_credit_active_binary'] = (bureau['CREDIT_ACTIVE'] != 'Closed').astype(int)
bureau['bureau_credit_enddate_binary'] = (bureau['DAYS_CREDIT_ENDDATE'] > 0).astype(int)
groupby_SK_ID_CURR = bureau.groupby(by=['SK_ID_CURR'])

In [ ]:
bureau.head()

In [ ]:
description[description['Row'] == 'DAYS_CREDIT'].Description.tolist()

In [ ]:
features = pd.DataFrame({'SK_ID_CURR':bureau['SK_ID_CURR'].unique()})
features.head()

In [ ]:
group_object = groupby_SK_ID_CURR['DAYS_CREDIT'].agg('count').reset_index()
group_object.rename(index=str, columns={'DAYS_CREDIT': 'bureau_number_of_past_loans'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

In [ ]:
group_object = groupby_SK_ID_CURR['CREDIT_TYPE'].agg('nunique').reset_index()
group_object.rename(index=str, columns={'CREDIT_TYPE': 'bureau_number_of_loan_types'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

In [ ]:
features['bureau_average_of_past_loans_per_type'] = \
    features['bureau_number_of_past_loans'] / features['bureau_number_of_loan_types']
    
features.head()

In [ ]:
group_object = groupby_SK_ID_CURR['bureau_credit_active_binary'].agg('mean').reset_index()

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

In [ ]:
group_object = groupby_SK_ID_CURR['AMT_CREDIT_SUM_DEBT'].agg('sum').reset_index()
group_object.rename(index=str, columns={'AMT_CREDIT_SUM_DEBT': 'bureau_total_customer_debt'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

In [ ]:
group_object = groupby_SK_ID_CURR['AMT_CREDIT_SUM'].agg('sum').reset_index()
group_object.rename(index=str, columns={'AMT_CREDIT_SUM': 'bureau_total_customer_credit'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

In [ ]:
features['bureau_debt_credit_ratio'] = \
    features['bureau_total_customer_debt'] / features['bureau_total_customer_credit']
    
features.head()

In [ ]:
group_object = groupby_SK_ID_CURR['AMT_CREDIT_SUM_OVERDUE'].agg('sum').reset_index()
group_object.rename(index=str, columns={'AMT_CREDIT_SUM_OVERDUE': 'bureau_total_customer_overdue'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

In [ ]:
features['bureau_overdue_debt_ratio'] = \
    features['bureau_total_customer_overdue'] / features['bureau_total_customer_debt']
    
features.head()

In [ ]:
group_object = groupby_SK_ID_CURR['CNT_CREDIT_PROLONG'].agg('sum').reset_index()
group_object.rename(index=str, columns={'CNT_CREDIT_PROLONG': 'bureau_total_prolonged_count'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

In [ ]:
group_object = groupby_SK_ID_CURR['bureau_credit_enddate_binary'].agg('mean').reset_index()
group_object.rename(index=str, columns={'bureau_credit_enddate_binary': 'bureau_credit_enddate_percentage'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

In [ ]:
X = application.merge(features,
                                left_on=['SK_ID_CURR'],
                                right_on=['SK_ID_CURR'],
                                how='left',
                                validate='one_to_one')

In [ ]:
engineered_numerical_columns = list(features.columns)
engineered_numerical_columns.remove('SK_ID_CURR')
X = X[engineered_numerical_columns + ['TARGET']]
X_corr = abs(X.corr())

In [ ]:
X_corr.sort_values('TARGET', ascending=False)['TARGET']

In [ ]:
sns.heatmap(X_corr, 
            xticklabels=X_corr.columns,
            yticklabels=X_corr.columns)

## Aggregations

In [ ]:
BUREAU_AGGREGATION_RECIPIES = [('CREDIT_TYPE', 'count'),
                               ('CREDIT_ACTIVE', 'size')
                               ]
for agg in ['mean', 'min', 'max', 'sum', 'var']:
    for select in ['AMT_ANNUITY',
                   'AMT_CREDIT_SUM',
                   'AMT_CREDIT_SUM_DEBT',
                   'AMT_CREDIT_SUM_LIMIT',
                   'AMT_CREDIT_SUM_OVERDUE',
                   'AMT_CREDIT_MAX_OVERDUE',
                   'CNT_CREDIT_PROLONG',
                   'CREDIT_DAY_OVERDUE',
                   'DAYS_CREDIT',
                   'DAYS_CREDIT_ENDDATE',
                   'DAYS_CREDIT_UPDATE'
                   ]:
        BUREAU_AGGREGATION_RECIPIES.append((select, agg))
BUREAU_AGGREGATION_RECIPIES = [(['SK_ID_CURR'], BUREAU_AGGREGATION_RECIPIES)]

In [ ]:
groupby_aggregate_names = []
for groupby_cols, specs in tqdm(BUREAU_AGGREGATION_RECIPIES):
    group_object = bureau.groupby(groupby_cols)
    for select, agg in tqdm(specs):
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        application = application.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(index=str,
                                      columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
        groupby_aggregate_names.append(groupby_aggregate_name)

In [ ]:
application.head()

In [ ]:
application_agg = application[groupby_aggregate_names + ['TARGET']]
application_agg_corr = abs(application_agg.corr())

In [ ]:
application_agg_corr.sort_values('TARGET', ascending=False)['TARGET']

# Solution 6
## Hand Crafted Features

In [ ]:
bureau['bureau_credit_type_consumer'] = (bureau['CREDIT_TYPE'] == 'Consumer credit').astype(int)
bureau['bureau_credit_type_car'] = (bureau['CREDIT_TYPE'] == 'Car loan').astype(int)
bureau['bureau_credit_type_mortgage'] = (bureau['CREDIT_TYPE'] == 'Mortgage').astype(int)
bureau['bureau_credit_type_credit_card'] = (bureau['CREDIT_TYPE'] == 'Credit card').astype(int)
bureau['bureau_credit_type_other'] = (~(bureau['CREDIT_TYPE'].isin(['Consumer credit',
                                                        'Car loan', 'Mortgage', 'Credit card']))).astype(int)
bureau['bureau_unusual_currency'] = (~(bureau['CREDIT_CURRENCY'] == 'currency 1')).astype(int)

In [ ]:
groupby_SK_ID_CURR = bureau.groupby(by=['SK_ID_CURR'])
group_object = groupby_SK_ID_CURR[['bureau_credit_type_consumer',
                                   'bureau_credit_type_car',
                                   'bureau_credit_type_mortgage',
                                   'bureau_credit_type_credit_card',
                                   'bureau_credit_type_other',
                                   'bureau_unusual_currency']].agg(['sum', 'mean']).reset_index()

group_object.columns = ['{}_{}'.format(a,b) for a,b in group_object.columns.tolist()]
group_object.rename(index=str, columns={'SK_ID_CURR_': 'SK_ID_CURR'},inplace=True)
features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
features.head()

In [ ]:
COLUMN_NAME = 'DAYS_CREDIT_UPDATE'
d = description[description['Row']==COLUMN_NAME]
display(d.Description.tolist())
display(d.Special.tolist())

In [ ]:
bureau.sort_values(['SK_ID_CURR','DAYS_CREDIT'],ascending=False, inplace=True)
bureau['days_credit_diff'] = bureau['DAYS_CREDIT'].diff().replace(np.nan, 0)    

groupby_SK_ID_CURR = bureau.groupby(by=['SK_ID_CURR'])

In [ ]:
group_object = groupby_SK_ID_CURR['bureau_credit_active_binary'].agg('sum').reset_index()
group_object.rename(index=str, columns={'bureau_credit_active_count': 'bureau_credit_active_binary'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')

In [ ]:
group_object = groupby_SK_ID_CURR['SK_ID_BUREAU'].agg('nunique').reset_index()
group_object.rename(index=str, columns={'SK_ID_BUREAU': 'bureau_query_count'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')

In [ ]:
def last_k_bureau_features(gr, periods):
    gr_ = gr.copy()

    features = {}
    for period in periods:
        if period > 10e10:
            period_name = 'all_records_'
            gr_period = gr_.copy()
        else:
            period_name = 'last_{}_'.format(period)
            gr_period = gr_[gr_['DAYS_CREDIT'] >= (-1) * period]

        features = add_features_in_group(features, gr_period, 'days_credit_diff',
                                             ['sum', 'min', 'max', 'median', 'std'],
                                        period_name)
        features = add_features_in_group(features, gr_period, 'CNT_CREDIT_PROLONG',
                                             ['sum', 'std'],
                                        period_name)
        features = add_features_in_group(features, gr_period, 'bureau_credit_active_binary',
                                             ['sum', 'mean'],
                                        period_name)
        features = add_features_in_group(features, gr_period, 'bureau_credit_type_consumer',
                                             ['sum', 'mean'],
                                        period_name)
        features = add_features_in_group(features, gr_period, 'bureau_credit_type_car',
                                             ['sum', 'mean'],
                                        period_name)
        features = add_features_in_group(features, gr_period, 'bureau_credit_type_credit_card',
                                             ['sum'],
                                        period_name)
        features = add_features_in_group(features, gr_period, 'bureau_credit_type_mortgage',
                                             ['sum'],
                                        period_name)
        features = add_features_in_group(features, gr_period, 'bureau_credit_type_other',
                                             ['sum', 'mean'],
                                        period_name)
    return features


def trend_in_last_k_bureau_features(gr, periods):
    gr_ = gr.copy()
    gr_['days_credit_diff'].iloc[0] = 0

    features = {}

    for period in periods:
        gr_period = gr_[gr_['DAYS_CREDIT'] >= (-1) * period]
        features = _add_trend_feature(features,gr_period,
                                      'days_credit_diff','{}_period_trend_'.format(period)
                                     )
    return features


def _add_trend_feature(features, gr ,feature_name, prefix):
    y = gr[feature_name].values
    try:
        x = np.arange(0,len(y)).reshape(-1,1)
        lr = LinearRegression()
        lr.fit(x,y)
        trend = lr.coef_[0]
    except:
        trend=np.nan
    features['{}{}'.format(prefix,feature_name)] = trend
    return features

In [ ]:
func = partial(last_k_bureau_features, periods=[60, 180, 360, 720, 1e25])

g = parallel_apply(groupby_SK_ID_CURR, func, index_name='SK_ID_CURR',
                   num_workers=10, chunk_size=10000).reset_index()
features = features.merge(g, on='SK_ID_CURR', how='left')

display(features.head())

In [ ]:
features.shape

In [ ]:
X = application.merge(features,
                      left_on=['SK_ID_CURR'],
                      right_on=['SK_ID_CURR'],
                      how='left',
                      validate='one_to_one')

In [ ]:
feature_names = list(features.columns)
feature_names.remove('SK_ID_CURR')
X = X[feature_names + ['TARGET']]
X_abs = abs(X.corr())

In [ ]:
X_abs.sort_values('TARGET', ascending=False)['TARGET']

In [ ]:
regex = 'credit_active'
X_corr_truncated = X_abs.sort_values('TARGET', ascending=False).filter(regex=regex, axis=0)
X_corr_truncated['TARGET']

In [ ]:
colnames = X_corr_truncated.index.tolist() + ['TARGET']
X_corr_truncated[colnames]

In [ ]:
sns.heatmap(X_corr_truncated[colnames], 
            xticklabels=colnames,
            yticklabels=colnames)